# NFT Example
### the input in time


$q(t)=\frac{2.2}{cosh(t)}$

### the input in &xi; domain
$c(\xi)=\frac{0.587783}{cosh(\pi\xi )} \cdot e^{-\pi j}$

### params:
* $t \in [-15:15] : (N_t=1024)$
* $\xi \in [-53:53] : (N_\xi=2048)$


### the INFT transform

$c(\xi) \overset{INFT}{\longrightarrow} \hat{q}(t) : q(t)=\hat{q}(t)$

#### params of the INFT
* "discretization": 4
* "contspec_type": 1
* "contspec_inversion_method": 0
* "discspec_type": 0
* "max_iter": 100
* "oversampling_factor": 8

# split step example
<h1><center>input signal</center></h1>
<center> $ \Large x = A_0 \cdot e^{-{\frac{1 - jC}{2}} \cdot (\frac{t}{t_0})^2} $ </center>
<br>
<center><table border="0">
 <tr>
    <td><h1>input params</h1></td>
    <td><h1>SSF params</h1></td>
 </tr>
 <tr>
    <td>
        $t \in  [-4096:4095] ps $ <br>
        $ N_t = 8192 $ <br>
        $ P_0 = 0.00064 $ <br>
        $ A_0 = \sqrt{P_0} $ <br>
        $ C = -2 $ <br>
        $ t_0 = 125ps $ <br>
    </td>
    <td>
        $ \beta_2 = -20*10^{-27} $ <br>
        $ \gamma = 0.003 $ <br>
        $ t_0 = 125ps $ <br>
        $ Z_N = 1.51 $ <br>
        $ dt = 1ps  $ <br>
        $ \Delta_z = 1000 $ <br>
    </td>
 </tr>
</table>
</center>

input signal specs:
- time spectral = [-0.32:0.32]*1e-9
- power = 7.93

## split step algorithm:

* $ w = [0,...,\frac{N_t}{2},\frac{-N_t}{2},...,-1] \cdot \frac{2 \pi}{N_t*dt}  $
* $ v_1 = exp(j\Delta_z\frac{\beta_2}{2} w^2) $
* $ \st{N = \frac{1}{\Delta_z}(1 + Z_N \cdot \frac{t_0^2}{\beta_2})} $
* $ N = \frac{Z_N}{\Delta_z} $
* $ a = x $
* loop N times:
    * $ a = \mathcal{F}(a) \cdot v_1 $
    * $ a = \mathcal{F}^{-1}(a) $
    * $ a = a*exp(j \gamma \Delta_z |a|^2) $
* return a

paper:
* $ L = T/dt $
* K is spatial (z) domain

* $ f = \frac{\beta_2}{2} (\frac{2\pi}{T})^2 (\frac{L}{2} - | \frac{L}{2} - l | )^2$
* $ D = exp(j \Delta_z f) $
* loop K times:
    * $ a = a * exp(j \gamma \Delta_z |a|^2 ) $
    * $ a = \mathcal{F}(a) $
    * $ a = a * D $
    * $ a = \mathcal{F}^{-1}(a) $

# TODO: what is K? is it equivalent to $ N = \frac{1}{\Delta_z}(1 + Z_N \cdot \frac{t_0^2}{\beta_2}) $ ?

## Channel Simulation
* msg
    * [modulation]
        * [oversample + pulse-shape]
            * [pre-equalize]
                * [INFT]
                    * [channel]
                * [INFT]
            * [equalize]
        * [match-filter]
    * [Demodulation]
* msg



# Channel Simulation

<center><table border="0">
 <tr>
    <td><h1>Simulator params</h1></td>
    <td><h1>SSF params</h1></td>
 </tr>
 <tr>
    <td>
        $ M_{QAM} = 16 $ <br>
        $ \text{num symbols: }N_s = 512 $ <br>
        $ \text{over sampling factor: }N_{os} = 8 $ <br>
        $ \text{normalization factor: }A_\mu = 10^{-3} $ <br>
        $ dt = 1ps $ <br>
    </td>
    <td>
        $ \alpha = 0 $ <br>
        $ \beta_2 = -20*10^{-27} $ <br>
        $ \gamma = 0.003 $ <br>
        $ dt = 1ps $ <br>
        $ T_0 = 125ps $ <br>
        $ \text{last step: }Z_N = 1 $ <br>
        $ h = 500 $ <br>
    </td>
 </tr>
</table>
</center>

### usual params:


### 0) input message:
we choose number of bits based on the number of symbols we want

so, number of bits: $ N_b = N_s*N_{sps} = 2048 $
where $ N_{sps} = log_2(M_{QAM}) = 4 $ is the num of samples per symbol

we get:
$ b=[b_0, b_1, ..., b_{N_b}] : b_i\in{0,1} \; \forall i$

### 1) Modulation
$ s = f_{MOD}(b) = [s_0, s_1, ... , s_{N_s}] : s_i = a_i + jb_i $

s.t:
$ f_{MOD}:B^{k*N_{sps}} \rightarrow C^k  $
$ N_{sps} = 512 $

### 2) over sampling

$ \hat s = [s_0, [0, 0, ..., 0], s_1, [0, 0, ..., 0], ..., s_{\hat N}, [0, 0, ..., 0]] $

s.t:

$ \hat s \in R^{\hat N} : \hat N = N_s*N_{os}= 4096 $


### 3) Pulse shaping
$ X[\xi] = h \circledast s  $

length of h, $N_h$ is chosen such that the length of X, $N_X$ will be the next closest power of 2
* (unless its too small [less than 100], so we skip to the next closest power of 2)

for $ \hat N = 4096 $ we chose $ N_h = 4097 $
and we get $ N_X = 8192 $

### 4) pre-equalize
$ \hat X(\xi) = X \cdot A_\mu$

### 5) INFT

$ x(t) = \mathcal{F}^{-1}\{\hat X\} $

with the following params:
* $ N_\xi = N_{sps} \cdot N_{os} = len(\hat X) = 8192 $
* $ N_t = 2 ^ {\lfloor log_2(N_\xi) \rfloor} $
* $ t = [-N_t / 2, N_t / 2] * dt $
* in: $ \xi ∈ [-1570.4:1570.8]$  GHz :<sp> $N_\xi  (=M) = 8192 $
* out: $ t  ∈ [-4096:4095]$ ps  <sp>:   $N_t    (=D) = 8192 $
* $ BW = 1570.8$ GHz

signal specs:
- time spectral = [-635:628]*1e-12
- signal power = 1721637827.78

### 6) SSF

now we go through the SSF channel
which has $N=1562$ iterations


$ Z_0 = T_0^2/|\beta_2|$ <br>
$ P_0 = 1/(\gamma Z_0) $ <br>
$ x = x *  \sqrt{P_0} $ <br>
$ \hat x = SSF(x) $ <br>
$ \hat{\hat x} = \hat x / \sqrt{P_0} $